# 20. Summary about Attempts

지금까지의 시도에 대한 생각들을 정리해둡니다.

## 지금까지 해온 것들

### Tic-Tac-Toe

Ch.01,03 에서는 Tic-Tac-Toe에 대해서
Q network를 학습시키거나 Monte Carlo 방식으로
win rate를 저장하는 tree를 만들어 보았습니다.

- 일단 Tic-Tac-Toe는 state space의 크기가 5478로
매우 작으며, action도 많아봐야 9개, 그 중 일부
symmetry나 비합리적인 수를 제외하면 끽해도 2-3개밖에 없습니다.
- 매우 작은 state/action space 때문에 그냥 가능한 경우를
모두 시뮬레이션해봐도 매우 짧은 시간 내에 답이 나옵니다.
- Monte Carlo method로 uniform random policy를 바탕으로
win rate tree를 완성시킬 때, 게임 1000번이면 전체
state space중 60% 가량을 커버하게 되고, 1만번 게임을
진행하면 약 97.6%를 커버하게 됩니다.
따라서 게임을 5000번 정도만 진행해도 충분히 잘 동작하는
agent를 얻을 수 있습니다.
- 위 숫자들이 말해주듯이, Q table을 만들어도 크기는
49302밖에 되지 않으므로 tabular method를 사용할 수 있습니다.
- Q table 대신 neural network를 사용한다고 하여도,
단순히 linear layer를 4층으로 쌓는 것만으로 Q-value를
학습하고도 남습니다.
- 이렇게 매우 적은 리소스로도 빠르게 결과를 볼 수 있기 때문에
테스트는 용이하나, 결과적으로 양측이 최선을 다하면 무조건
무승부가 나는 게임이기 때문에 얼마나 잘 학습되는지는
알 수 없습니다.

## Mock5.py

오목을 학습 시키기 전, environment가 필요하다 판단하였고,
다른 사람이 만든 것보다는 직접 만들어서 쓰는 것이 뜯어고치기
유리하다고 생각하여 `mock5.py`라는 모듈을 만들었습니다.
(https://github.com/lumiknit/mock5.py)
계속 학습을 시도하면서 일부 필요한 것이 있으면 기능을 추가하였는데,
Ch.02,15 에서는
학습 상대로 필요한 `agent_analysis_based`를 비롯한
`agent_random`, `agent_ad`, `agent_df`, `agent_pt`,
그리고 이것들의 구현에 필요한 `analysis` 모듈을 구현했고,
Ch.04,06 에서는
기보를 빠르게 많이 뽑아내기 위해 기보 생성기를 C++과 OpenMP로
만들었습니다.
이후에는 위 agent를 섞은 `agent_mixed`를 주 스파링 상대로써
학습을 진행하였습니다.

### Omok with Value Network

Ch.05-09는 value network를 바탕으로 mock5.py용 agent를
만드는 것이 목표였고, 실패하였습니다.

- Ch.01을 하고 나서는 그 구조를 그대로 오목에 적용하면 안 될까 하여 시도를 해보았는데 (https://github.com/lumiknit/mock5-q)
학습이 전혀 이루어지지 않았습니다. 지금 보면 택도 없이
작은 network에 짧은 학습시간에 뭐가 되기를 바라던 것 같습니다.
- 이 시점에서는 선행연구나 구현체에 대한 참고 없이 무작정
만들다보니 network 크기나 학습 시간을 과감히 키우지 않았던
것 같습니다. 다만 Ch.09에서는 CNN 4층으로 약간은 오목을
진행할 정도로는 학습이 되는 것은 확인하였습니다.
- `mock5-q`에서는 Q function을 network로 학습하려고 했지만,
(1) 오목은 이전의 action들과 무관하게 현재 판의 상태가
승률을 결정한다; (2) action이 많아지면 학습해야 하는 경우의 수가
늘어서 학습이 오래걸린다; 라는 이유로 V function을 학습하는
쪽으로 방향을 바꾸게 되었습니다.
- 학습 속도를 올리기 위해서 비교적 모범답안을 제시해주는 기보
생성기를 주로 활용하게 되었고,
이 때문에 학습이 supervised learning에 가까웠습니다.

### Connect 4

Value network만으로는 학습이 제대로 진행이 되어서
다른 강화학습 기법이나 AlphaGo/AlphaZero 관련 자료를 찾아보았습니다.
그 때 실제로 network를 훨씬 깊게 쌓아야 한다든가
Connect 4도 학습시간이 상당히 오래 걸린다는 것을 듣고는
일단은 오목보다는 더 단순한 Connect4에서 학습을 진행하고자 했습니다.

Ch.12-14가 Connect 4를 학습하는 과정에 해당합니다.

- Connect 4는 게임 끝나는 것을 무시한다고 할 때 가능한 
state의 수가
$14007180988362844601443040716800$
$\simeq 1.401 \times 10^{31} \simeq 1.38 \times 2^{103}$
개 입니다.
판 크기가 오목의 절반도 안 되지만, 이미 대부분 CPU의 정수크기는
아득히 넘어갑니다.
- 오목이랑은 다르게 state의 shape와 action의 shape가 다릅니다.
이 때문에 CNN만으로 network를 쌓아서 바로 action을 가져오는
것은 힘들고, 마지막에 linear layer 등으로 aggregation을
계산해야 합니다.
- 여기서는 CNN 4층, dense 2층으로 network를 구성하였습니다.
- 제일 처음에는 이전처럼 Q function을 근사해보고자 했지만,
마찬가지로 잘 학습되지 않았습니다.
- 두번째로 MCPG (REINFORCE)를 적용해보았고,
처음으로 아주 가끔 policy network가 이기는 것을 확인할 수 있었습니다.
- 이후 임의의 policy가 있을 때 tree search를 바탕으로
더 성능이 좋은 policy transformer를 만드는 것을 시도하였습니다.
- 학습 과정을 손봐서 승리한 사람의 state-action만을
데이터로 샘플하고,
판을 좌우로 뒤집은 경우도 학습 데이터로 넣음으로써
학습 속도를 놀라울 정도로 끌어올릴 수 있었습니다.
(처음에는 약 4만번의 게임을 해도 1% 미만의 승률을 보여주지만
이후에는 1만번의 게임으로 100%에 가까운 승률을 보여주었습니다.)
- 학습 속도 덕분에 학습 상대를 바꿔가면서 비교를 해볼 수 있었는데,
학습 과정 자체가 '이긴 사람의 행동에 대한 확률을 강화'시키는
방향으로 이루어지다보니, 상대가 강하면 강할수록 더 강한 수를
배워가는 것을 볼 수 있었습니다.
또한 policy network를 바탕으로 stochastic하게 동작하다보니
exploration을 설정할 필요 없이 최선의 수를 찾아가려고 합니다.
- 상대가 base knowledge가 없는 자기 자신이나 random일
경우에는 최선의 수를 찾는데 한참 방황을 해서 학습이 굉장히
느립니다. `agent_greedy` 정도로 나름 무난하게 두지만
그리 강하지 않은 상대가 있으면 빠르게 행동을 따라하다가
막상막하가 되었을 때부터 상대를 이기기 위해 exploration을
하고 더 강화하는 모습을 보여줍니다.
만약 상대가 이보다 더 강하면 상대의 다양한 대응 때문에 학습
속도가 느리기는 하지만, 더 착실하게 실력을 올려가는 것을
확인할 수 있습니다.


### Omok with Policy Gradient

Ch.15-19에서는 policy gradient를 바탕으로 policy network
를 학습시킵니다.

- Policy network는 각 위치별로 확률이 필요하다보니
입력 shape와 출력 shape가 거의 같습니다.
이 때문에 dense layer를 쓰면 위치 정보가 사라질 수 있어
neural network 전체를 CNN으로 구현합니다.
- 한편 CNN이면 신경망과 입력의 크기가 무관하기 때문에
오목판의 크기도 $15 \times 15$로 늘려서 학습합니다.
- Ch.15-16에서는 Connect4에서 썼던 방법을 그대로 오목에 적용합니다.
약 1시간 동안 15000번가량의 게임을 진행해서 `agent_mixed`와
상대해서 20% 가량의 승률을 거두는 것이 가능하다는 것을 확인했습니다.
- Ch.17에서는 A2C를 구현해보았는데,
그리 좋은 결과를 얻지 못했습니다.
- Ch.18에서는 AlphaZero와 비슷하게 MCTS를 사용해보았는데,
학습이 안 되는 것은 아니었지만, 병렬처리 등의 도움 없이는
게임 진행이 너무 느리고, 이로 인해 학습이 너무 느려져서
포기하였습니다.
- Ch.19에서는 neural network에 residual block을 일부
더하고 feature dimension을 늘린 채로 10시간 정도 학습을
진행하였고, 10만번의 게임을 끝내고 나서 제일 처음에 만들었던
`agent_analysis_based` (`aa`)를 상대로 이기는 것에
성공했습니다.

## 보드게임 강화학습과 관련하여

강화학습을 하면서 각 파트별로 들었던 생각들을 정리합니다.


### Value-based Learning vs Policy-based Learning

강화학습은 결국에는 최적의 정책 $\pi^*$를 얻는 것이 목표이고,
이를 적당히 근사한 정책 $\pi$를 구하는 것이 주된 흐름입니다.
그런데 많은 경우에 action-value function
$q$를 최대로 하는 것이 좋은 정책이 되다보니
$\pi(s) = \text{argmax}_{a \in A} q(s, a)$
와 같이 둔 채로 $q$를 근사하는 것도 가능합니다.
여기서 $q$를 근사하는 것이 value-based,
$\pi$를 근사하는 것이 policy-based가 됩니다.

저 차이 때문에 학습할 때 고려할 것이 조금 다릅니다.

- 다른 환경에서는 모르겠지만, 보드게임에서는 보상이 승/패로
주어지다보니, $q$는 state-action에 대한 기대 승률이
됩니다.
- $q$는 특정 값들끼리 수렴시키는 과정이다보니 $L_2$-loss
를 최소화하는 방향으로 학습해나가게 됩니다.
- 반대로 policy gradient를 비롯한 policy-based learning은
$\pi$에 따라 행동을 해서 얻는 보상의 크기만큼 해당 행동을
강화시키도록 학습하는데, 이 때문에 classification처럼
negative log loss를 최소화하도록 학습합니다.
- $q$를 직접 policy로 사용하게 되면, exploration에 대한
고려가 중요해집니다.
최종적으로 학습이 완료되었다면 `argmax`처럼 greedy하게
행동하면 되겠지만, 중간중간에 게임을 진행하며 더 강해지려면
자신이 지금까지 생각한 최선의 수보다 더 좋은 수가 있는지
탐색을 해야하는데, 오목처럼 action space가 너무 큰 경우에는
랜덤하게 탐색하는 것은 비효율적이므로 $q$값을 바탕으로 각
수를 정렬해서 차선책을 택하는 등 방법을 써야합니다.
- $\pi$를 사용하게 되면 애초에 policy 자체가 stochastic하기
때문에 exploration에 대한 고민은 그다지 하지 않아도 좋습니다.
다만, 간단한 REINFORCE 알고리즘 등은 on-policy learning이기
때문에 replay 등을 적용하는 것이 불가능합니다.
- $\pi$를 직접 학습할 경우 생기는 부수적인 문제인데,
classification에서 데이터의 라벨을 log 앞에 곱하는 것과
달리, 여기서는 최종적으로 얻는 return이나 $q$ 등을
라벨 대신 곱하게 됩니다.
이 return이 음수인 경우에 문제가 될 수 있는데,
만약 낮은 확률의 행동을 우연히 고르게 되면 $\log$ 값이
매우 커질 수 있고, 이 때 return마저 음수라면 gradient가
경사를 내려가는 반대방향으로 미친듯이 커질 수 있습니다.
이것이 몇번 누적되면 그대로 weight가 우주로 날아가서
`inf`나 `NaN`을 보는 일이 발생할 수 있습니다.
- $q$가 승률을 나타낸다면 승/패 무관하게 기보 전체를 받아서
값을 갱신해야되지만,
$\pi$는 이긴 사람의 수만 학습하는 것으로도 괜찮은 결과를 볼 수
있습니다.
- 오목과 같이 action은 많다만 어떤 식으로 탐색해야될지 어렵고
단순히 승률의 문제가 아닌 수읽기 등을 통한 전략이 있다든가
episode가 충분히 짧은 경우에는 REINFORCE를 제일 먼저
적용해보는 것이 좋다고 생각합니다.

### Neural Networks

- 오목의 policy network 등에 생각보다 깊은 신경망이 필요하지는
않습니다.
AlphaGo가 CNN을 약 13층 가량 쌓아서 만들어졌다고 알고 있는데,
Connect 4는 CNN 4층에 dense layer 2층으로도 알고리즘을 이길
수 있고, 오목은 CNN 8층으로 알고리즘을 이길 정도의 복잡도는 됩니다.
- Dense layer는 복잡한 함수를 정확하게 근사하는 것에 도움이
될지는 모르지만, 각 점에 대한 위치정보가 전부 소실될 수 있습니다.
따라서 학습을 할 때 각 위치에 대한 action을 다 해야만 빈틈없이
학습을 하는 것이 가능할텐데, self-play만으로 학습한다면 이를
조율하는 것이 쉽지 않을 것이라고 생각합니다.
- 오목은 각 행동을 할 확률이 전역적인 정보보다는 그 칸 근처의
지역적인 정보로 결정되는 편이다보니, dense layer 없이
convolutional layer만 사용해도 좋은 결과를 얻을 수 있습니다.
- Kernel의 크기도 그다지 중요하지 않았습니다.
여기서는 kernel size 3으로 8개의 convolutional layer를
쌓으면 반경 8칸을 덮는 것이 가능한데, 이 정도면 정 중앙의
$3 \times 3$개 칸에서 $15 \times 15$ 칸의 가장자리까지
모두 덮어지는 크기입니다.
오목은 공격이 정사각형 범위를 넘어서 영향을 미치기 매우 힘들다보니
지역적인 정보로도 충분한 것 같습니다.
- 단, 지역적인 정보들이 뭉쳐서 사라지면 제대로된 결과를 얻지
못할 수 있어서, feature dimension은 높게 유지할
필요가 있습니다.
Ch.19에서는 feature dimension을 128로 유지했지만,
policy network만으로 잘 학습시키려면 256정도는 되어야 할 것
같습니다.
- 강화학습에서 batch normalization과 dropout은 그리
유의미하지 않습니다.
우선 batch normalization은 학습할 때 적당한 크기의
minibatch가 모여야 써먹을 수 있는데,
특히 on-policy learning,
temporal difference learning은 minibatch가 매우 작아져서
normalize할 크기가 나오지 않습니다.
dropout도 overfit을 방지하기 위해서 사용하는데,
오목 같이 state나 action space가 너무 큰 경우에는
차라리 overfit이 나면 좋겠을 정도로 underfit이 잘 일어나는
듯 하며, 일부 perceptron이 꺼지면 제대로된 판단이
불가능할 수 있어서 사용하지 않습니다.
- 강화학습이 빠르게 이루어지려면 안정적으로 수렴시키는 것보다는
작은 batch로 방향을 빠르게 흔들어서 local optimum을 빠르게
탈출하고 강하하는 것이 중요하다고 생각하는데,
ReLU를 사용하는 경우에는 이 방식대로 빠르게 흔들면
ReLU가 과다하게 죽어버리는 일이 발생하는 것 같습니다.
이 때문에 ReLU 대신 다른 activation을 사용해보았는데,
GELU가 이 문제가 제일 덜한 것으로 보입니다.
아마 swish나 mish 등도 비슷하게 괜찮을 것으로 생각됩니다.
- 확률이나 승률을 뽑아내기 위해서 neural network 마지막단에
$\tanh$, $\sigma$, $\text{softmax}$를 사용하게 되는데,
셋 모두 값이 일정 범위를 넘어가게 되면 함수값이나 미분값이
상수나 0이 되는 문제가 있습니다.
이렇게 되면 학습이 진행되지 않거나, 다양할 선택지를 고를
확률이 0으로 가는 등의 문제가 있습니다.
이 부분은 학습 중간에 weight가 너무 심하게 튀지 않도록
조절해서 해결할 수밖에 없어 보입니다.


### Training Data

- Policy gradient니 MCTS니 뭐니 해도
결국에는 policy network와 value network를
기보를 바탕으로 학습을 하기 때문에 **기보의 수가 곧 학습된 agent의 퀄리티가 됩니다.**
- 원래는 on-policy 학습을 하면 자신이 마지막으로 한
기보는 완전히 버리고 새로운 policy에 맞춰서 게임을 진행하도록
해야합니다만, 실제로는 재활용해도 크게 문제가 생기지는 않습니다.
- 마찬가지로 판을 회전시킬 경우에 policy에 따른 action은
단순히 회전시킨 것이 아니라 완전히 다를 수도 있는데,
이러한 변환을 적용한 것들을 몽땅 일괄적으로 학습해도 큰 문제는
생기지 않습니다.
- Minibatch가 너무 커지게 되면 학습의 방향이 안정적인 방향으로
 될
수는 있는데, 이 경우에 오히려 local optimum을 탈출하지 못해서
성능이 나아지지 않습니다.
- 바둑판을 입력으로 줄 때에는 보통은 one-hot encoding으로
주게 되는데,
빈칸을 위해 따로 dimension을 주는 것은 그다지 필요 없는 것
같습니다.
빈 칸을 `(0, 0)`, 자신의 돌을 `(1, 0)`, 상대의 돌을 `(0, 1)`
로 encoding하는 것으로도 충분해보입니다.



### Optimization

- Value network의 경우라면 몰라도, policy gradient를
학습시킬 때에는 안정적인 학습보다 global optimum을 찾는
것이 우선시 되므로 Adam을 쓰는 것이 크게 도움이 안 될 수도
있습니다.
비슷한 이유에서 momentum을 높게 잡는 것이 전반적으로
도움이 됩니다.
- 일반적인 initialization으로는 학습이 잘 안 될 수도 있습니다.
그래서 아예 적당한 범위에서 uniform random으로 weight
initialization을 해주거나, 조금 큰 learning rate로
일부 input을 넣어 weight를 흔들어 주는 것이 도움이 되는 것
같습니다.
- Policy gradient는 자주하는 행동을 반복하거나, 안 하던
좋지 않은 행동을 반복하면 weight가 비정상적으로 커질 수 있습니다.
따라서 weight decay를 빼놓지 않는 것이 좋습니다.

### MCTS

- MCTS는 크게 2가지 면에서 도움을 주는데,
하나는 매우 정확하게 근사되지 않은 policy function과
value function에 대해서 정확도를 수읽기로 보완할 수 있다는
것이며,
다른 하나는 단순히 policy/value function만 사용할 경우보다
더 좋은 수를 제공하여서 학습의 질을 높인다는 것입니다.
- 반대로 MCTS로 근사를 하게 되면 network를 여러번 실행을
해야합니다. (Ch.18에서는 한 수를 둘 때 100번 가량
network를 사용합니다.)
이 때문에 게임 진행이 그만큼 늦어지게 되며, 이는 학습속도
저하로 직결됩니다.
- 실제로 Ch.18에서 약 8시간동안 1700번의 게임이 진행되었는데,
Ch.19에서 같은 시간이면 8만번의 게임이 진행되어 매우 큰
차이를 보여줍니다.

### Others

- 위에서도 여러번 말한 것처럼 기보의 수가 곧 성능인 만큼,
가장 확실하게 학습 속도를 올리는 방법은 같은 시간 동안 더 많은
게임을 할 수 있게 최적화를 하는 것입니다.
예를 들어서 Mock5.py 에서 돌을 둘 때마다 확인하는 코드 등으로
오버헤드가 존재하는데, 기보 생성기를 사용하여 이를 상당히 줄이는
것이 가능했습니다.
- 비슷하게 학습 과정의 몇몇 부분을 병렬화 하는 것을 생각해볼 수
있는데, off-policy 학습을 할 경우에는 simulation을 여러
policy에 대해 병렬적으로 실행한 다음 학습을 진행할 수도 있을
것이고, MCTS에서 트리를 탐색 및 확장하는 것을 병렬처리 한다든가
여러 agent가 한 게임에 대해 병렬적으로 탐색하는 등등
여러 지점이 있습니다.
- 결국 이런 문제는 CPython 위에서 돌아가는 현 환경에서는
해결하기 힘들고, 가능하면 lua 위에서 실행되는 Torch나,
아예 PyTorch C++ Frontend, Tensorflow for C++로
넘어가는 것이 제일 좋아보입니다.


## 더 해볼만한 것들

- Feature engineering:
원래는 돌의 연결상태 등을 추가적인 feature로 전달하면
성능이 더 좋아지지 않을까 생각했습니다만,
이것을 찾아내는 것 자체가 병렬화하지 않고서는 매우 느리다보니
적용하지 않게 되었습니다.
- Threat Space Search(TSS), Proof Number Search(PNS):
TSS는 1993년 L.V.Allis, H.J.Herik, M.P.H.Huntjens의
'Go-Moku and Threat-Space Search'에서 나오는데,
간단히 말하자면 오목에서 공격을 해서 게임을 끝낼 수 있는 공간을
threat space라고 하고, tree search를 할 때 해당 구역만을
탐색 범위로 제한하여 속도를 올리겠다는 것입니다.
한편 PNS는 and-or-tree에서의 best-first
tree search algorithm입니다.
얼마 전까지만 해도 많이 쓰이던 알고리즘이라고 하니 이 역시
알아보면 좋지 않을까 싶습니다.

Copyright (c) 2022 Hyogun Lee (lumiknit, aasr4r4@gmail.com)